# Section 1.3: Accuracy and the Receiver Operating Characteristic (ROC) Curve

Just how accurate is your machine-learning (ML) model? The question might seem trivial at first: just look at the percentage of predictions or classifications that it gets right. But examining only the accuracy score of an ML model tells only part of the story of its usefulness in the real world. All models produce false positives and false negatives; which of those should you tune your model to favor? If your model is to be used to identify individuals carrying an infectious disease, fewer false negatives at the cost of more false positives could be desirable. If you are making classifications in support of criminal cases, accepting more false negatives to avoid false positives might be essential.

> **Learning objective:** By the end of this section, you should have a basic understanding of the different means of assessing the accuracy of an ML model and how the ROC curve can help you understand their relationship in your own models.

## Accuracy

To assess the accuracy of a model, we need to create and fit one first.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Load and prepare the data

We'll again use the same dataset drawn from the [U.S. Department of Agriculture National Nutrient Database for Standard Reference](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/nutrient-data-laboratory/docs/usda-national-nutrient-database-for-standard-reference/) that you used in Sections 1.1 and 1.2. (Note that the path name is case sensitive.)

In [ ]:
df = pd.read_csv('Data/USDA-nndb-combined.csv', encoding='latin_1')

Our model in this section doesn't like `NaN` values any more than principal-component analysis (PCA) did in Section 1.2, so we'll drop all rows with `NaN`s in them.

> **Exercise**
>
> Drop all rows containing `NaN` values in `df`. (**Hint:** Refer to this page if you are unsure about which method to use to perform this action.)

In [ ]:
df = df.dropna()

As we did in Section 1.2, let's divide our current dataset into descriptive and quantitative `DataFrame`s.

In [ ]:
desc_df = df.iloc[:, [0, 1, 2]+[i for i in range(50,54)]]
desc_df.set_index('NDB_No', inplace=True)
desc_df.head()

In [ ]:
nutr_df = df.iloc[:, :-5]
nutr_df = nutr_df.drop(['FoodGroup', 'Shrt_Desc'], axis=1)
nutr_df.set_index('NDB_No', inplace=True)
nutr_df.head()

Let's take a closer look at `nutr_df`. We'll use the `DataFrame` `describe()` method to do this, but, given the number of columns, we'll also want to transpose the `DataFrame` using the `T` attribute.

In [ ]:
nutr_df.describe().T

The `describe()` method provides a wealth of information about the `DataFrame`. For our purposes, it's particularly useful for identifying those columns whose means are much larger or much smaller than their medians (their respective 50th percentiles). Such differences indicate features in which a minority of comparatively large or small instances skew the means.

In particular, several foods appear to be very low in calories, so much so that the mean of `Energ_Kcal` is close to its 25th percentile level, almost a full quartile away from where we might expect it, closer to its median value. As a result, `Energ_Kcal` will be the feature we'll later train our model to identify.

Correlated features are still an issue, so we'll remove the `Folate_DFE_(µg)`, `Vit_A_RAE`, and `Vit_D_IU` columns.

In [ ]:
nutr_df.drop(['Folate_DFE_(Âµg)', 'Vit_A_RAE', 'Vit_D_IU'], 
        inplace=True, axis=1)
nutr_df.head()

### Prepare and fit the model

Before we can fit a model to find low-calorie foods, we first must define numerically for the model what we mean by "low-calorie." We do this by means of a dummy variable, in this case, a new column that reads `1` if the value in a row in `Energ_Kcal` is less than equal to the mean for that column and `0` otherwise.

In [ ]:
nutr_df['Low_cal'] = np.where(nutr_df['Energ_Kcal']<=nutr_df['Energ_Kcal'].mean(), 1, 0)
nutr_df.head()

Pandas creates our new column at the end of the `DataFrame`.

Now to fit the model. Drawing from conventions established by the statistical community for regression models, we'll refer to our predictor features (otherwise known as the independent variables) as `X` and our response (the outcome or dependent variable) as `y`. Because we want to predict `Low_cal`, we cannot have it in `X`. Similarly, because `Low_cal` is really just a Boolean alias for `Energ_Kcal`, we should also remove `Energ_Kcal` from `X`. Finally, because the fat content of foods is generally so closely correlated with foods' energy content, we should also not include `Lipid_Tot_(g)` in `X`. (We want our model to be good but not too good to illustrate the ROC curve later in the section.)

> **Technical note**
>
> Using capital `X` for the predictors and lower-case `y` for the response is a widely used convention in statistics and data science that we will use here and elsewhere in these courses.

We'll set `y` to just be equal to `Low_cal`.

In [ ]:
nutr_df.dropna()

### Prep the data

Let's create two `DataFrame`s, ones to contain the data, and another for the value we want to predict.

In [ ]:
X = nutr_df.drop(['Energ_Kcal', 'Low_cal', 'Lipid_Tot_(g)'], axis=1)
y = nutr_df['Low_cal']

Now randomly split the data between training and test datasets. Scikit-learn's `train_test_split` is particularly convenient for this; use the `test_size` parameter to reserve 30 percent of the data for testing (and 70 percent for training). The `random_state` parameter ensures that training and test data split the same way each time so that these results are reproducible.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=67)

Finally, it's time to fit the model against the training data and get predictions from it against the test data.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)
predictions = lr.predict(X_test)

### Assess the accuracy of the model

Scikit-learn provides three means of assessing the accuracy of models: the accuracy score, the confusion matrix, and the classification report.

Why so many ways of measuring accuracy for a model? Recall from the beginning of the section that success means different things in different contexts. In some cases, avoiding false negatives is paramount (such as when working with infectious diseases). In other cases, avoided false positives is essential (such as in instances involving the criminal-justice system). Other times, striking a balance of false positives and negatives is the best approach. These different measures provided the tools for data scientists to best tune their models for the problems at hand.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#### Accuracy score

Our accuracy score tells us the fraction of correctly classified samples: the sum of the number of correct true and false predictions divided by the number of all predictions.

In [ ]:
accuracy_score(y_test, predictions)

#### Confusion matrix

The confusion matrix is another way to present this same information, this time with raw scores. The columns show the true condition, with positive on the left and negative on the right. The rows show predicted conditions, with positive on the top and negative on the bottom. So, the matrix below shows that our model correctly predicted 258 low-calorie foods (true positives) and incorrectly predicted another 23 (false positives). On the other hand, our model correctly predicted 371 higher-calorie foods (true negatives) and incorrectly predicted 5 more (false negatives).

In [ ]:
pd.DataFrame(confusion_matrix(y_test, predictions), 
             columns=['True low cal', 'True high cal'], 
             index=['Predicted low cal', 'Predicted high cal'])

The classification reports the proportions of low-calorie and high-calorie foods with four scores:
 - **Precision:** The number of true positives divided by the sum of true positives and false positives; closer to 1 is better.
 - **Recall:** The true-positive rate, which is the number of true positives divided by the sum of the true positives and the false negatives.
 - **F1 score:** The harmonic mean (the average for rates) of precision and recall.
 - **Support:** The number of true instances for each label.

In [ ]:
print(classification_report(y_test, predictions))

# 0 is Low Cal
# 1 is High Cal

> **Takeaway:** This subsection gave you a taste of performing classification using logistic regression by removing extraneous variables, checking for multicollinearity, handling missing values, and fitting and evaluating your model. In the next subsection, we'll look at visualizing the accuracy of ML models.

## ROC Curve

The ROC curve was developed during World War II to measure the ability of radar-receiver operators to correctly identify aircraft, hence the name, "receiver operating characteristic." From military application, the ROC curve made its way more broadly to signal-detection theory and from there to statistics and data science.

ROC curves typically plot the true positive rate on the Y axis and the false positive rate on the X axis. This means that the top-left corner of the plot is the “ideal” point: a false positive rate of 0 (zero), and a true positive rate of 1. Such an ideal is rarely (if ever) realized in practice, but it is the theoretical goal.

A key concept with ROC curves is *area under the curve* (AUC), with a larger AUC representing a better model. The slope of the ROC curve is also important because it's better to maximize the true positive rate while minimizing the false positive rate.

Let's find the ROC curve and AUC for the model we just trained.

In [ ]:
from sklearn.metrics import roc_curve, auc

In order to graph the ROC curve, we need to find the true positive rate (TPR) and false positive rate (FPR) at all thresholds of the classification.

In [ ]:
probs = lr.predict_proba(X_test)
print(probs)

The `predict_proba()` function returns estimates for both classes (high-calorie and low-calorie) for every instance in `X_test`. Because each outputted pair of numbers adds up to 1, we really only need to retain one half of each pair. We'll go with the low-calorie probabilities and we can redefine `probs` to just contain those.

In [ ]:
probs = probs[:,1]

The `roc_curve()` function then compares the actual values (`0` or `1`) from `y_test` against the probabilities generated by the model (`probs`) and returns the FPR, the TPR, and the thresholds (the instances of prediction by the model).

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, probs)

In [ ]:
fpr

In [ ]:
tpr

The `auc()` function computes the area under the ROC curve for our FPR and TPR values.

In [ ]:
rocauc = auc(fpr, tpr)

In [ ]:
rocauc

Our model has an AUC of 0.99, which is higher than our accuracy rate of 0.96. Not only is our model accurate, but it achieves high rates of true positive classifications with very low rates of false positive classifications.

Let's see this graphically.

In [ ]:
plt.title('Receiver Operating Characteristic (ROC) metric')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % rocauc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

> **Takeaway:** Because no ML model is perfect, it's important to know a model's TPR and FPR. The ROC curve (and the AUC figure) provide a concise means of summarizing a lot of nuance about your model's performance.